In [54]:
!pip install pycuda ThrustRTC scikit-cuda

In [55]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [56]:
import sys
import os
path = "/content/drive/My Drive"
sys.path.append(path)
os.chdir(path)
%cd graduation\ project/

/content/drive/My Drive/graduation project


In [57]:
!nvidia-smi

Wed Oct  6 18:34:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    31W / 250W |   1330MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [58]:
import numpy as np
import pandas as pd
from skcuda import cublas
from pycuda import gpuarray
import pycuda.autoinit
import time
from pycuda.compiler import SourceModule
import pycuda.driver as cuda
import ThrustRTC as trtc
import numpy as np
import time
from matplotlib import pyplot
import pandas as pd
from sklearn import metrics
from collections import Counter
import random

FLOAT_MAX = 1e10

In [59]:
# 求平方
get_square = SourceModule(
'''
__global__ void x_square(float *x, int &n)
{
    const int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if(idx<n)
        x[idx] = x[idx] * x[idx];
}

__global__ void y_square(float *y, int &m)
{
    const int idx = threadIdx.x;
    if(idx<m)
        y[idx] = y[idx] * y[idx];

}
'''
)

# 求和
square_sum = SourceModule(
'''

__global__ void sum_row_xy(float *s_x, float *s_y, float *self_sum1, float *self_sum2, int &numSample, int &k, int &dim)
{
    const int idx = threadIdx.x + blockIdx.x * blockDim.x;
    
    if(idx < k)
        for(int i=0;i<dim;i++)
            self_sum2[idx] += s_y[idx*dim+i];


    if(idx < numSample + k && idx >= k)    
        for(int j=0;j<dim;j++)    
            self_sum1[idx - k] += s_x[(idx - k)*dim+j];    
    
    __syncthreads(); 
}

__global__ void sum_total(float *self_sum1, float *self_sum2, float *result, int &k, int &numSample)
{
    const int idx1 = threadIdx.x + blockIdx.x * blockDim.x;
    int quo = idx1 % numSample;
    int rem = idx1 / numSample;
    
    if(idx1 < numSample * k)
    
        result[idx1] = self_sum1[quo] + self_sum2[rem];

    __syncthreads();     

}


'''
)


# shared_memory.max()=48KB(12000 int/float32)
find = SourceModule(
'''
__global__ void find_range(float *cluster, float *result)
{
    __shared__ float sdata[512];
    unsigned int tid = threadIdx.x;
    unsigned int i = blockIdx.x*blockDim.x+ threadIdx.x;
    sdata[tid] = cluster[i];
    __syncthreads();
    if(sdata[tid] != sdata[tid+1] and tid < 499) // 统计各簇中元素的个数
        result[int(sdata[tid])] = i+1;
    __syncthreads();
}

'''
)

find_minimum = SourceModule(
'''
__global__ void find_min(float *list, float *result_value, int *result_idx, int &length, int &n)
{
    __shared__ float sdata[1024];
    __shared__ int ssdata[1024];
    unsigned int tid = threadIdx.x;
    unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;   


    if(tid < length-1)
    {
        sdata[tid] = list[i]; // store data
        ssdata[tid] = tid; // store index
    }
    sdata[length - 1] = 10000;
    ssdata[length - 1] = 23;  

    __syncthreads();
    
    unsigned int s;
    if(tid < length)
    {
        for(s = (blockDim.x+1)/2; s>0; s>>=1) {
            if (tid < s) 
            {
                ssdata[tid] = (sdata[tid] <= sdata[tid + s])? ssdata[tid]:ssdata[tid+s];
                __syncthreads();
                sdata[tid] = (sdata[tid] <= sdata[tid + s])? sdata[tid]:sdata[tid+s];
                __syncthreads();
            }

            __syncthreads();

}
        if(s == 0)
        {
            ssdata[0] = (sdata[0] <= sdata[n-1])? ssdata[0]:ssdata[n-1];
            sdata[0] = (sdata[0] <= sdata[n-1])? sdata[0]:sdata[n-1];
            __syncthreads();
        }    
    }
       
    if(tid == 0)
    {
        result_idx[blockIdx.x] = ssdata[0];
        result_value[blockIdx.x] = sdata[0];     
    }

}


'''
)


def cluster_assignment_new(C, k):

    n = k    
    divider = k + 1 if(k % 2) else k        
    while(divider % 2 == 0):
        n = divider / 2
        divider /= 2
    n = np.int32(n)
    n_gpu = cuda.mem_alloc(n.nbytes)
    cuda.memcpy_htod(n_gpu, n)
    
    length = np.int32(k+1)
    length_gpu = cuda.mem_alloc(length.nbytes)
    cuda.memcpy_htod(length_gpu, length)
    result_value = gpuarray.to_gpu(np.zeros((len(C)//k,),dtype = np.float32))
    result_idx = gpuarray.to_gpu(np.zeros((len(C)//k,),dtype = np.int32))
    C = gpuarray.to_gpu(C)

    find_func_hybrid = find_minimum.get_function("find_min")
    find_func_hybrid(C, result_value, result_idx, length_gpu, n_gpu, block=(k, 1, 1),grid=(len(C) // k, 1))
    cluster_assignment = np.zeros((len(result_idx),2))
    
    cluster_assignment[:,0] = result_idx.get()
    cluster_assignment[:,1] = result_value.get()
    return cluster_assignment

In [60]:
# matrix1 is dataset, matrix2 is centroids
def get_x_square(matrix1):
    """
      It can be interpolated that the dataset square need to calculate for only once!
    """
    n = matrix1.shape[0] * matrix1.shape[1]
    grid_dim = int(np.ceil(n / 1024))
    n = np.int32(n)
    n_gpu = cuda.mem_alloc(n.nbytes)
    cuda.memcpy_htod(n_gpu, n)

    x = matrix1.flatten().astype(np.float32)
    
    x_gpu = gpuarray.to_gpu(x)
    x_func = get_square.get_function("x_square")
    
    # we have a flexible block assignment
    x_func(x_gpu, n_gpu, block=(1024,1,1), grid=(grid_dim,1))
    return x_gpu # could be used as matrix1 in get_xy_square

#数据点得分批去输
def get_xy_square(x_gpu, matrix2, cluster_k, numSample, dim):
    m = cluster_k * dim
    # determining grid size
    m = np.int32(m)
    m_gpu = cuda.mem_alloc(m.nbytes)
    cuda.memcpy_htod(m_gpu, m)
    grid_dim = int(np.ceil(m / 1024))

    y_gpu = matrix2.astype(np.float32)

    # y_gpu = gpuarray.to_gpu(y)
    y_func = get_square.get_function("y_square")
    
    y_func(y_gpu, m_gpu, block=(1024,1,1), grid=(grid_dim,1))

    self_sum1 = np.zeros((numSample, ), dtype = np.float32)
    block_dim = len(self_sum1) // 1024 + 1
    self_sum2 = np.zeros((cluster_k, ), dtype = np.float32)
    
    self_sum1_gpu = gpuarray.to_gpu(self_sum1)
    self_sum2_gpu = gpuarray.to_gpu(self_sum2)
    
    numSample_np = np.int32(numSample)
    numSample_gpu = cuda.mem_alloc(numSample_np.nbytes)
    cuda.memcpy_htod(numSample_gpu, numSample_np)
    
    k = np.int32(cluster_k)
    k_gpu = cuda.mem_alloc(k.nbytes)
    cuda.memcpy_htod(k_gpu, k)
    
    dim = np.int32(dim)
    dim_gpu = cuda.mem_alloc(dim.nbytes)
    cuda.memcpy_htod(dim_gpu, dim)

    result = np.zeros((cluster_k*numSample, ), dtype = np.float32)
    block_dim_result = len(result) // 1024 + 1
    
    result_gpu = gpuarray.to_gpu(result)
   
    # optimization v1.0
    sum_func_row = square_sum.get_function('sum_row_xy')

    grid_dim_new = 500000 // 1024
    sum_func_row(x_gpu, y_gpu, self_sum1_gpu, self_sum2_gpu, numSample_gpu, k_gpu, dim_gpu, block = (1024,1,1), grid = (grid_dim_new,1))
    
    sum_func_total = square_sum.get_function("sum_total")
    sum_func_total(self_sum1_gpu, self_sum2_gpu, result_gpu, k_gpu, numSample_gpu, block=(1024,1,1), grid=(block_dim_result,1))
    
    return result_gpu # result_gpu could be used for matrix3 in dis_computation


# 计算与质心之间的距离
def dis_computation(matrix1, matrix2, matrix3, orishape_matrix1):
    """matrix 1 is centroids, 
    matrix 2 is dataset.transpose, 
    matrix 3 is result"""
    
    matrix1 = matrix1.astype(np.float32)
    matrix2 = matrix2.astype(np.float32)
    
    ldc = matrix2.shape[1]
    m = matrix2.shape[1]
    n = orishape_matrix1[0]

    #C = matrix3.flatten()
    #the above is matrix C
    ldb = matrix2.shape[0]
    k = matrix2.shape[0]
    B_gpu = matrix1
    #the above is matrix B

    lda = matrix2.shape[1]
    A_gpu = matrix2
    # the above is matrix A
    transa = 'n'
    transb = 'n'
    alpha = -2
    beta = 1
    #copy data into GPU
    # A_gpu = gpuarray.to_gpu(A)
    # B_gpu = gpuarray.to_gpu(B)
    #C_gpu = gpuarray.to_gpu(C)
    # computing matrix
    h = cublas.cublasCreate()
    cublas.cublasSgemm(h, transa, transb, m, n, k, alpha, A_gpu.gpudata, lda, B_gpu.gpudata, ldb, beta, matrix3.gpudata, ldc)
    cublas.cublasDestroy(h)
    
    C = matrix3.get()
    C = np.reshape(C, (n, m))
    C = C.transpose() # 1484 * 10 or 494021 * 5
    C = C.flatten()

    return C


def cluster_assignment(C, k=23):
# 实质上这里也是在排序
    if(len(C) == 1):
        return np.array([C,0])
    C = C.reshape((494021,k))
    minDist = np.amin(C, axis = 1)
    minDistIdx = np.argmin(C, axis = 1)
    cluster_assignment = np.vstack((minDistIdx, minDist)).T
    return cluster_assignment



def sort_by_key_without_count(clusterAssment):
    cluster = clusterAssment[:,0].astype(np.float32)
    cluster_gpu = trtc.device_vector_from_numpy(cluster)
    n = cluster_gpu.size()
    index = trtc.device_vector("int32_t", n)
    trtc.Sequence(index)
    trtc.Sort_By_Key(cluster_gpu, index)
    index_cpu = index.to_host()
    cluster_cpu = cluster_gpu.to_host()
    return index_cpu, cluster_cpu


def sort_gpu(index_cpu, cluster_cpu, dataset, k, placement):
    cluster1 = cluster_cpu[0:250000].astype(np.float32)
    cluster2 = cluster_cpu[250000:-1].astype(np.float32)
    result1 = np.zeros((k-1,),dtype=np.float32)
    result2 = np.zeros((k-1,),dtype=np.float32)
    find_func = find.get_function("find_range")
    find_func(cuda.In(cluster1),cuda.Out(result1), block=(500,1,1),grid=(500,1))
    find_func(cuda.In(cluster2),cuda.Out(result2), block=(500,1,1),grid=(500,1))
    # 取置信值，这里的代码可以优化一下
    # 如果数据量控制在25w以内效果最佳
    for j in range(len(result2)):
        if(result2[j] != 0):
            result2[j] += 250000 # 分批处理，第一批计数从0开始，第二批从250000开始
    for i in range(len(result2)-1):
        if(result2[i] > result2[i+1]): # 消除异常值
            result2[i] = 0
    starting_points = [0]
    for jj in range(len(result2)):
        starting_points.append(max(result1[jj], result2[jj]))
    starting_points.append(dataset.shape[0]+1)
    data_in_cluster = []
    start = int(starting_points[placement])
    end = int(starting_points[placement+1])
    data_in_cluster = dataset[index_cpu[start:end],:]
    data_in_cluster = np.array(data_in_cluster)
    return data_in_cluster



def cluster_assignment_gpu(C):
    C = C.astype(np.float32)
    length = np.int32(C.shape[1])
    length_gpu = cuda.mem_alloc(length.nbytes)
    cuda.memcpy_htod(length_gpu, length)
    result = np.zeros((C.shape[0],), dtype = np.float32)
    # 找到距离最近的质心点
    find_func = find.get_function("find_minimum")
    find_func(cuda.In(C),cuda.Out(result),length_gpu,block=(C.shape[1],1,1),grid=(C.shape[0],1))
    
    result_idx = np.argmin(C, axis = 1)    
    cluster_assignment = np.zeros((len(result_idx),2))
    cluster_assignment[:,0] = result_idx
    cluster_assignment[:,1] = result
    return cluster_assignment


In [61]:
import pycuda

# cpu的版本
def euclDistance(vector1,vector2):
    return np.sqrt(sum(np.power(vector2-vector1,2)))#power计算次方

##初始化数据的中心点，k表示聚类中心数
##随机生成k个聚类中心
def initCentroids(dataset,k):
    numSample,dim=dataset.shape
    centroids=np.zeros((k,dim)).astype(np.float32)
    for i in range(1,k+1):
        #index=int(np.random.uniform(0,numSample))#随机生成数
        index = int(i*10000)
        centroids[i-1,:]=dataset[index,:]
    return centroids

##kmean算法
def kmeans(dataset,k):
    pycuda.tools.clear_context_caches()
    numSample, dim = dataset.shape
    #生成新的两列数组，保存聚类信息
    # 第一列表示所属聚类中心，第二列表示与中心的误差
    clusterAssment = np.zeros((numSample,2))#这里dtype就默认
    clusterChanged = True

## step1 初始化聚类中心
    centroids=initCentroids(dataset,k)
    storage = np.ones((numSample,))
    storage = storage * k
    itr = 0
    dataset_transpose_gpu = gpuarray.to_gpu(dataset.T)
    dataset_gpu_square = get_x_square(dataset)
    orishape_matrix1 = centroids.shape

    while (clusterChanged):
        itr += 1
        count = 0
        for cnt in range(len(storage)):           
            if(storage[cnt] != clusterAssment[cnt,0]):
                count += 1
                if(count > 10):
                    clusterChanged = True
                    break
            else:
                clusterChanged = False
        
        storage = clusterAssment[:,0]
        #二重循环：对所有数据点，与k个聚类中心计算距离
        #并保存标签与距离
        matrix1 = gpuarray.to_gpu(centroids.flatten())
        matrix2 = dataset_transpose_gpu
        matrix3 = get_xy_square(dataset_gpu_square, matrix1, k, numSample, dim)
        distance_mat = dis_computation(matrix1, matrix2, matrix3, orishape_matrix1)
        # matrix1.gpudata.free()
        
        clusterAssment1 = clusterAssment # buffer
        clusterAssment = cluster_assignment_new(distance_mat, k)


## step4 循环结束后更新聚类中心
        #index, cluster, count_dict = sort_by_key(clusterAssment)
        #index, cluster = sort_by_key_without_count(clusterAssment)
        #collection = []
        for i in range(k):
            comp = np.nonzero(clusterAssment[:,0] == i)[0] # 当前状态的聚类情况
            comp1 = np.nonzero(clusterAssment1[:,0] == i)[0] # 上一状态的聚类情况
            if(len(comp) == len(comp1) and (comp == comp1).all()):
                 # 当前簇内元素没有改变？如果没有改变，下次聚类忽略该簇
                continue
                
            #pointsInCluster = sort_gpu(index, cluster, dataset, k, i) 
            points_In_k_Cluster_Label=np.nonzero(clusterAssment[:,0]==i)[0]
            pointsInCluster=dataset[points_In_k_Cluster_Label]
            centroids[i, :] = np.mean(pointsInCluster, axis=0)
            #centroids[i,:] = centroids_update(pointsInCluster, k)
        
        if(itr == 30):            
            break
    print("Iteration time is %.d" % itr)
    
    ##循环结束，返回聚类中心和标签信息
    return centroids,clusterAssment

In [63]:
if __name__=="__main__":
    # print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    # start=time.time()
    ## load data
    dataset=pd.read_csv('kdd_pre_final.csv',sep=',')
    # 真实的标签
    category_real = dataset.loc[:,["classification"]]
 
    dataset=dataset.iloc[:,:-1] 
    dataset=np.array(dataset)
    
    ##  k表示聚类中心数
    k = 23
    total_time = 0
    times = 5
    for _ in range(times):
        start=time.time()
        centroids,clusterAssment=kmeans(dataset,k)
        end = time.time()
        total_time += (end - start)
    print('algorithm (for training) average time: %.2f seconds'%(total_time / times))
    
    category_real = np.array(category_real)
    category = []
    for i in range(dataset.shape[0]):
        category.append(category_real[i][0])
    category = np.array(category)
    category_pre = np.array(clusterAssment[:,0], dtype = np.int32)
    real = Counter(category)
    pre = Counter(category_pre)
    print(real)
    print(pre)
    real = real.most_common()
    pre = pre.most_common()
    for j in range(dataset.shape[0]):
        for nn in range(k):
            if(category[j] == real[nn][0]):
                category[j] = int(pre[nn][0])
    ARI = metrics.adjusted_rand_score(category, category_pre)
    AMI = metrics.adjusted_mutual_info_score(category, category_pre)
    print("The ARI value is {:.4f}".format(ARI))
    print("The AMI value is {:.4f}".format(AMI))

Iteration time is 28
Iteration time is 28
Iteration time is 28
Iteration time is 28
Iteration time is 28
algorithm (for training) average time: 6.52 seconds
Counter({5.0: 280790, 4.0: 107201, 0.0: 97278, 13.0: 2203, 15.0: 1589, 10.0: 1247, 9.0: 1040, 20.0: 1020, 8.0: 979, 7.0: 264, 17.0: 231, 6.0: 53, 1.0: 30, 11.0: 21, 19.0: 20, 14.0: 12, 22.0: 10, 2.0: 9, 12.0: 8, 18.0: 7, 16.0: 4, 3.0: 3, 21.0: 2})
Counter({12: 253482, 11: 48529, 8: 38218, 5: 38215, 13: 37371, 10: 20457, 9: 11896, 2: 10479, 22: 8007, 19: 6941, 1: 5816, 7: 5393, 3: 5098, 0: 2316, 15: 1243, 14: 294, 21: 53, 17: 48, 4: 46, 6: 43, 20: 35, 18: 24, 16: 17})
The ARI value is 0.7499
The AMI value is 0.7345
